# <center> Inference </center>

In this notebook we will show how to perform inference of our Neuralpoisson model. In this case we are loading the input point cloud by performing marching cubes on voxel grids (.binvox files) representing shapes from the [ShapeNetsem dataset](https://shapenet.org/download/shapenetsem).

## Load libraries

In [1]:
# Utilities from dave
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from skimage.filters import threshold_otsu
import plotly.graph_objects as go
from skimage.transform import resize

In [2]:
sys.path.append('../src/')
from importlib import reload
import utils
reload(utils)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<module 'utils' from '/scratch/Github_repos/Neuralpoisson/examples/../src/utils.py'>

## Loading model

We first load the model

In [3]:
width = 30
modes = 10
device = torch.device('cuda')
neuralpoisson = utils.NeuralPoisson(modes, modes, modes, width).cuda()

Now we load the checkpoint corresponding to the sampling rate

In [4]:
# n_samples = 3000
# sigma = 2

# n_samples = 10000
# sigma = 1.5

# n_samples = 25000
# sigma = 1.0

n_samples = 250000
sigma = 0.5

In [5]:
checkpoint = '../checkpoints/checkpoints_n_samples_'+str(n_samples)

neuralpoisson.load_state_dict(torch.load(checkpoint))
neuralpoisson.eval()

NeuralPoisson(
  (fc0): Linear(in_features=67, out_features=30, bias=True)
  (conv0): SpectralConv3d()
  (conv1): SpectralConv3d()
  (conv2): SpectralConv3d()
  (conv3): SpectralConv3d()
  (w0): Conv3d(30, 30, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (w1): Conv3d(30, 30, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (w2): Conv3d(30, 30, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (w3): Conv3d(30, 30, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  (bn0): BatchNorm3d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1): BatchNorm3d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm3d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm3d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=30, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)

## Loading data

All data from the subfolder `ShapeNetSem/models-binvox-solid/` should be placed in the `data/`

In [6]:
path = '../data/fe1081843a08b7a64d8fdc4d7658fe42.binvox'

In [15]:
data = utils.gen_batch_example(path, 250000, 1.2, lowres = True)
divergence_tensor_batch = data[0]
voxels_batch = data[1]
points_batch = data[2]
normals_batch = data[3]
path_batch = data[4]

In [17]:
# The method needs a batch with 2 elements to work
divergence_tensor_batch[0] = divergence_tensor_batch[1]
voxels_batch[0] = voxels_batch[1]
points_batch[0] = points_batch[1]
normals_batch[0] = normals_batch[1]
path_batch[0] = path_batch[1]

In [18]:
# Computing prediction
pred_batch = utils.Neuralpoisson_predict_highres(divergence_tensor_batch, neuralpoisson, batch_size=2)

/scratch/torch_envs/shaperecon_clean/lib/python3.9/site-packages/torch/nn/functional.py:1967: UserWarning:

nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.



In [19]:
# Thresholding
voxels = voxels_batch[0]
divergence_tensor = divergence_tensor_batch[0]
path = path_batch[0]
points = points_batch[0]
normals = normals_batch[0]
pred = pred_batch[0]
pred_bin = 1-utils.binarize(pred[0], threshold_otsu)

In [20]:
# Save ground truth and reconstruction
utils.save_results_benchmarks(voxels, pred_bin, points, path)

Saved results


## Visualizing results

In [21]:
utils.visualize_pred(voxels)

In [22]:
utils.visualize_pred(pred_bin)

In [19]:
## Uncomment if you want to visualize the pointcloud
# x, y, z = points.T
# u, v, w = normals.T
# fig = utils.init_figure()
# tr = go.Cone(
#     x=x, y=y, z=z, u=u, v=v, w=w, colorscale='Blues', sizemode="absolute", sizeref=2)
# fig.add_trace(tr)